In [ ]:
import pandas as pd
import requests
import datetime

#Крупные транзакции за последние Х минут

In [ ]:
import requests
import datetime

SOLANA_RPC_URL = "https://api.mainnet-beta.solana.com"

#### 1
# Функция получения информации о блоках за последние N секунд
def get_recent_blocks(seconds=None):
    current_slot = requests.post(SOLANA_RPC_URL, json={
        "jsonrpc": "2.0", "id": 1, "method": "getSlot"
    }).json()['result']

    block_time = requests.post(SOLANA_RPC_URL, json={
        "jsonrpc": "2.0",
        "id": 1,
        "method":
        "getBlockTime",
        "params": [current_slot]
    }).json()['result']

    start_time = int(datetime.datetime.utcnow().timestamp()) - seconds
    past_slot = current_slot

    while block_time and block_time > start_time:
        past_slot -= 1
        block_time = requests.post(SOLANA_RPC_URL, json={
            "jsonrpc": "2.0",
            "id": 1,
            "method":
            "getBlockTime",
            "params": [past_slot]
        }).json().get('result', None)
        if block_time is None:
            break
    return list(range(past_slot, current_slot))


#### 2
# Функция получения списка транзакций в формате JSON
def get_transactions_from_blocks(blocks):
    transactions_list = []
    solusd_price = requests.get("https://api.coingecko.com/api/v3/simple/price?ids=solana&vs_currencies=usd").json()['solana']['usd']
    for block in blocks:
        response = requests.post(SOLANA_RPC_URL, json={
            "jsonrpc": "2.0",
            "id": 1,
            "method": "getBlock",
            "params": [block, {"encoding": "json",
                               "transactionDetails": "full",
                               "maxSupportedTransactionVersion": 0}]
        }).json()

        if "result" in response:
            for tx in response["result"]["transactions"]:
                meta = tx.get("meta", {})
                message = tx["transaction"]["message"]
                account_keys = message["accountKeys"]
                pre_balances = meta.get("preBalances", [])
                post_balances = meta.get("postBalances", [])
                block_time = response["result"]["blockTime"]

                if len(account_keys) >= 2 and len(pre_balances) >= 2 and len(post_balances) >= 2:
                    from_address = account_keys[0]
                    to_address = account_keys[1]
                    amount = round((post_balances[1] - pre_balances[1])/10**9*solusd_price, 4)

                    transactions_list.append({
                        "block_id": block,
                        "trans_id": tx["transaction"]["signatures"][0],
                        "block_time": block_time,
                        "time": datetime.datetime.utcfromtimestamp(block_time).isoformat() + "Z",
                        "activity_type": "ACTIVITY_SPL_TRANSFER",
                        "from_address": from_address,
                        "to_address": to_address,
                        "token_decimals": 9,
                        "amount": amount,
                        "flow": "in" if amount > 0 else "out"
                    })

    return {"success": True, "data": transactions_list}

#Получатель - отправитель - поиск токена - изменение баланса кошелька

Покупка токена
adress = token
sender = user
reciever = token


Продажа токена
adress = user
sender = toker
recivier = user

In [ ]:
import requests
import json
import time
import pandas as pd
import ast

solusd_price = requests.get("https://api.coingecko.com/api/v3/simple/price?ids=solana&vs_currencies=usd").json()['solana']['usd']

### 4
# Функция для получения информации об аккаунте (кошелек, токен-аккаунт, контракт и т.д.)
def get_account_info(public_key):
    response = requests.post(SOLANA_RPC_URL, json={
        "jsonrpc": "2.0",
        "id": 1,
        "method": "getAccountInfo",
        "params": [public_key, {"encoding": "jsonParsed"}]
    }).json()

    value = response.get("result", {}).get("value", {})
    return value.get("owner", "Unknown")

### 5
# Функция для получения информации о токене
def get_token_info(mint_address):
    response = requests.post(SOLANA_RPC_URL, json={
        "jsonrpc": "2.0",
        "id": 1,
        "method": "getAccountInfo",
        "params": [
            mint_address,
            {"encoding": "jsonParsed"}
        ]
    }).json()

    token_info = response.get("result", {}).get("value", {}).get("data", {}).get("parsed", {}).get("info", {})
    return {
        "token_name": token_info.get("symbol", "Unknown"),
        "token_address": mint_address
    }


### 5
# Функция для получения данных о транзакции
def get_transaction_wallets(transaction_signature, solusd_price):
    headers = {"Content-Type": "application/json"}
    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "getTransaction",
        "params": [transaction_signature,
                  {"encoding": "jsonParsed",
                   "maxSupportedTransactionVersion": 0}]}

    for attempt in range(3):
        response = requests.post(SOLANA_RPC_URL, json=payload, headers=headers)
        data = response.json()
        if "error" in data and data["error"].get("code") == 429:
            print(f"⚠️ Слишком много запросов (429). Ждем 3 секунды... Попытка {attempt+1}/3")
            time.sleep(5)
            continue
        if "result" not in data or not data["result"]:
            return []

        transaction_data = data["result"]
        meta = transaction_data.get("meta", {})
        transaction_info = transaction_data.get("transaction", {})
        #print(transaction_info)

        if not meta or "preBalances" not in meta or "postBalances" not in meta:
            return []

        account_keys = [str(key) for key in transaction_info.get("message", {}).get("accountKeys", [])]
        pre_balances = meta["preBalances"]
        post_balances = meta["postBalances"]
        token_transfers = meta.get("postTokenBalances", [])
        sender = account_keys[0] if len(account_keys) > 0 else "Unknown"
        receiver = account_keys[1] if len(account_keys) > 1 else "Unknown"



        token_map = {}
        for token in token_transfers:
            token_address = token.get("mint")
            owner_address = token.get("owner")
            ui_amount = token.get("uiTokenAmount", {}).get("uiAmount", 0)
            if isinstance(owner_address, str) and token_address:
                token_info = get_token_info(token_address)
                token_info["amount"] = ui_amount
                token_map[str(owner_address)] = token_info

        rows = []
        change_in_per = []
        alerts = []

        for i in range(len(account_keys)):
            account_key = account_keys[i]

            account_type = get_account_info(account_key)
            token_info = token_map.get(account_key, {"token_name": "SOL", "token_address": "So11111111111111111111111111111111111111112", "amount": (post_balances[i] - pre_balances[i]) / 1e9})

            # пробуем получить имя токена
            #address
            try:
              get_token_data = requests.get(f"https://api.dexscreener.com/latest/dex/pairs/{'solana'}/{ast.literal_eval(account_key)['pubkey']}").json()
              address_token_name = get_token_data['pairs'][0]['baseToken']['symbol']
            except:
              address_token_name = 'address_token_not_found'
            #sender
            try:
              get_token_data = requests.get(f"https://api.dexscreener.com/latest/dex/pairs/{'solana'}/{ast.literal_eval(sender)['pubkey']}").json()
              sender_token_name = get_token_data['pairs'][0]['baseToken']['symbol']
            except:
              sender_token_name = 'sender_token_not_found'
            #reciever
            try:
              get_token_data = requests.get(f"https://api.dexscreener.com/latest/dex/pairs/{'solana'}/{ast.literal_eval(receiver)['pubkey']}").json()
              reciever_token_name = get_token_data['pairs'][0]['baseToken']['symbol']
            except:
              reciever_token_name = 'reciever_token_not_found'

            rows.append({
                "transaction_signature": transaction_signature,
                "address": ast.literal_eval(account_key)['pubkey'],
                "address_token_name": address_token_name,
                "pre_balance": float(pre_balances[i] / 1e9),
                "post_balance": float(post_balances[i] / 1e9),
                "change_amount": float((post_balances[i] - pre_balances[i]) / 1e9),
                "change_in_USD": round(float((post_balances[i] - pre_balances[i]) / 1e9 * solusd_price), 4),
                "change_in_%": round(float((post_balances[i] - pre_balances[i]) / pre_balances[i] * 100 if pre_balances[i] > 0 else 0), 4),
                "token_amount": token_info["amount"],
                "sender": sender,
                "sender_token_name":sender_token_name,
                "reciever_token_name":reciever_token_name,
                "receiver": receiver
            })
            if ast.literal_eval(account_key)['pubkey'] == ast.literal_eval(receiver)['pubkey']:
               change_in_per = [round(float((post_balances[i] - pre_balances[i]) / pre_balances[i] * 100 if pre_balances[i] > 0 else 0), 4)]

            if address_token_name != 'address_token_not_found':
              if float(change_in_per[0]) > 0:
                alert = f"Wallet {ast.literal_eval(receiver)['pubkey']} did transaction {token_info['amount']} {address_token_name} ($={round(float((post_balances[i] - pre_balances[i]) / 1e9 * solusd_price), 4)}) and changed balance on {change_in_per[0]} %"
                print(alert)
                alerts.append(alert)
        return rows, alerts

    return []


### 6
# Функция для обработки списка транзакций
def get_multiple_transactions(transaction_signatures):
    all_rows = []
    all_alerts = []

    for signature in transaction_signatures:
        #print(f"🔍 Обрабатываем транзакцию: {signature} ...")
        transaction_rows, alerts = get_transaction_wallets(signature, solusd_price)
        all_rows.extend(transaction_rows)
        all_alerts.extend(alerts)
        time.sleep(3)

    return pd.DataFrame(all_rows), all_alerts


#MVP

In [ ]:
all_alerts = []

In [ ]:
### 3
# Получаем список активных транзакций
blocks = get_recent_blocks(1000)  # последние 10 минут
transactions = get_transactions_from_blocks(blocks)

print(len(transactions['data']))

transactions_list = []
for i in transactions['data']:
  if i['amount'] > 100:
    print('finded = $', round(i['amount'], 4), '_in_ b=',i['block_id'], i['trans_id'])
    transactions_list.append(i['trans_id'])

# Получаем DataFrame с данными
transaction_df1, alerts = get_multiple_transactions(transactions_list)

7859
finded = $ 145.145 _in_ b= 358842334 4yUUnUsEMGzEiV7GJFYQXzPe3ng1gahV1VpFzBrmDDGzkT3w8kPwdSxNaifus3xh6zoiEib9jk69UQRuWgmaf8Fc
finded = $ 145.145 _in_ b= 358842334 59FFZcZTD3o8YGFLQJ54D1X8khnS1UVAReZQj44oE2Gv1XM8G6hepzfRHnkyHyUYwDDdcDU4yYGMZXa5mB4UP4Tx
finded = $ 145.145 _in_ b= 358842334 26H6ye5R6D7WdfCrcFDaPuW3YfgD181tFxP17kfBPu5YpStxbjVsMeawJqQzSvkGLpWdJAfkH5GxJgkSGNb5M3LG
finded = $ 255.6793 _in_ b= 358842334 5t9x4DxH1uybj2SfCUYhmzgaKCX3Uo6PeFF4rhn7xxskfap6apKzEEoK3eBHiCpQxFyapAUw1fVwJcgjSWyz9bao
finded = $ 184.876 _in_ b= 358842334 3acsMsEQRWBpVrPNjoFksqV2wE2mgm4t4n9iJYxgaWuw3qBHR4vFcoEFDY3dEmqsCQHma6w1ic8hoc6qiHGiNi7i
finded = $ 145.145 _in_ b= 358842334 2jSaP6QaGhCAvgLkchrkjtZ3TX8WRDwEWoh3tVD1u7fK7h7SNCJSBCa8a8MFCnKbmrLyJEBjY8BGhvYa4MC9t1rW
finded = $ 169.2054 _in_ b= 358842334 61xqv4DTjLDD5EPEy9Jxq6AwYFHd5C2PJd5doVs1XnExNzow5BQCU65Q9SsAY82pffV6FsJECD9sKVneNHa91Db3
finded = $ 145.145 _in_ b= 358842334 m78e78oVJVXxagQviUvwX3bCRy4cNUki394sBeHuuL2G1crX7Q2J7jtnQnGXUETAkwW82dBnHM

KeyboardInterrupt: 

In [ ]:
all_alerts.extend(alerts)
all_alerts

['Wallet 6t43fBYRihjwhoHurxarDTv6aFcbGbADwaedn9QZYYAk did transaction 0.0 Grok  ($=0.0) and changed balance on 0.7324 %']

In [ ]:
transaction_df1

,transaction_signature,address,address_token_name,pre_balance,post_balance,change_amount,change_in_USD,change_in_%,token_amount,sender,sender_token_name,reciever_token_name,receiver
0,4xdLPmspBV5yyQHeUcE2EDH4fvvuAhFv97LFCSN8HSYxqc...,Gdqg7Lzf9q1KcLf5hMGbd2yn8i1Si3pWpX3tFgpQtpcB,address_token_not_found,1.919851e+01,1.797011e+01,-1.228398,-152.7391,-6.3984,-1.228398,{'pubkey': 'Gdqg7Lzf9q1KcLf5hMGbd2yn8i1Si3pWpX...,sender_token_not_found,reciever_token_not_found,{'pubkey': 'Gf4M16JSPe4goGvGcjhsD7eXVrD6v5pc47...
1,4xdLPmspBV5yyQHeUcE2EDH4fvvuAhFv97LFCSN8HSYxqc...,Gf4M16JSPe4goGvGcjhsD7eXVrD6v5pc47M5o5F9idqe,address_token_not_found,0.000000e+00,1.228393e+00,1.228393,152.7384,0.0000,1.228393,{'pubkey': 'Gdqg7Lzf9q1KcLf5hMGbd2yn8i1Si3pWpX...,sender_token_not_found,reciever_token_not_found,{'pubkey': 'Gf4M16JSPe4goGvGcjhsD7eXVrD6v5pc47...
2,4xdLPmspBV5yyQHeUcE2EDH4fvvuAhFv97LFCSN8HSYxqc...,11111111111111111111111111111111,address_token_not_found,1.000000e-09,1.000000e-09,0.000000,0.0000,0.0000,0.000000,{'pubkey': 'Gdqg7Lzf9q1KcLf5hMGbd2yn8i1Si3pWpX...,sender_token_not_found,reciever_token_not_found,{'pubkey': 'Gf4M16JSPe4goGvGcjhsD7eXVrD6v5pc47...
3,351ptZNcJX3kLURosXv8xQbAttbTowGMFH9Thnyuq1vu9S...,5ZdYLdMDTBMajTSyz7DyfMiiCdGJ3x4AgvH3JghjyAGJ,address_token_not_found,2.178279e+01,2.031867e+01,-1.464118,-182.0485,-6.7214,-1.464118,{'pubkey': '5ZdYLdMDTBMajTSyz7DyfMiiCdGJ3x4Agv...,sender_token_not_found,reciever_token_not_found,{'pubkey': '7pNpBa7GfSab69kAvThQjjdhTudjkvpFgN...
4,351ptZNcJX3kLURosXv8xQbAttbTowGMFH9Thnyuq1vu9S...,7pNpBa7GfSab69kAvThQjjdhTudjkvpFgN9TatpqQ5wP,address_token_not_found,0.000000e+00,1.452039e+00,1.452039,180.5466,0.0000,1.452039,{'pubkey': '5ZdYLdMDTBMajTSyz7DyfMiiCdGJ3x4Agv...,sender_token_not_found,reciever_token_not_found,{'pubkey': '7pNpBa7GfSab69kAvThQjjdhTudjkvpFgN...
5,351ptZNcJX3kLURosXv8xQbAttbTowGMFH9Thnyuq1vu9S...,61n6BkxihjZZwwWnYiGHEH3Aw3XWef3tb6UeAZU4xLFJ,address_token_not_found,0.000000e+00,1.000000e-02,0.010000,1.2434,0.0000,0.010000,{'pubkey': '5ZdYLdMDTBMajTSyz7DyfMiiCdGJ3x4Agv...,sender_token_not_found,reciever_token_not_found,{'pubkey': '7pNpBa7GfSab69kAvThQjjdhTudjkvpFgN...
6,351ptZNcJX3kLURosXv8xQbAttbTowGMFH9Thnyuq1vu9S...,Bf6FyELmN8gS5SpKFHegkYeqVfe2FksdTNBEdKepymb1,address_token_not_found,0.000000e+00,2.074080e-03,0.002074,0.2579,0.0000,0.002074,{'pubkey': '5ZdYLdMDTBMajTSyz7DyfMiiCdGJ3x4Agv...,sender_token_not_found,reciever_token_not_found,{'pubkey': '7pNpBa7GfSab69kAvThQjjdhTudjkvpFgN...
7,351ptZNcJX3kLURosXv8xQbAttbTowGMFH9Thnyuq1vu9S...,ATokenGPvbdGVxr1b2hvZbsiqW5xWH25efTNsLJA8knL,address_token_not_found,7.319136e-01,7.319136e-01,0.000000,0.0000,0.0000,0.000000,{'pubkey': '5ZdYLdMDTBMajTSyz7DyfMiiCdGJ3x4Agv...,sender_token_not_found,reciever_token_not_found,{'pubkey': '7pNpBa7GfSab69kAvThQjjdhTudjkvpFgN...
8,351ptZNcJX3kLURosXv8xQbAttbTowGMFH9Thnyuq1vu9S...,So11111111111111111111111111111111111111112,address_token_not_found,9.981705e+02,9.981705e+02,0.000000,0.0000,0.0000,0.000000,{'pubkey': '5ZdYLdMDTBMajTSyz7DyfMiiCdGJ3x4Agv...,sender_token_not_found,reciever_token_not_found,{'pubkey': '7pNpBa7GfSab69kAvThQjjdhTudjkvpFgN...
9,351ptZNcJX3kLURosXv8xQbAttbTowGMFH9Thnyuq1vu9S...,11111111111111111111111111111111,address_token_not_found,1.000000e-09,1.000000e-09,0.000000,0.0000,0.0000,0.000000,{'pubkey': '5ZdYLdMDTBMajTSyz7DyfMiiCdGJ3x4Agv...,sender_token_not_found,reciever_token_not_found,{'pubkey': '7pNpBa7GfSab69kAvThQjjdhTudjkvpFgN...
